In [1]:
import os
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

In [ ]:
from glob import glob
from tqdm.auto import tqdm

import wandb

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import ModelNet
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MLP, PointConv, fps, global_max_pool, radius

In [2]:
wandb.init(project="pyg-point-cloud", name="modelnet40-test", entity="geekyrakshit", job_type="eda")

config = wandb.config
config.display_sample = 2048

# Classes for ModelNet40
categories = sorted([
    x.split(os.sep)[-2]
    for x in glob(os.path.join("ModelNet40", "raw", '*', ''))
])


config.categories = categories

wandb: Currently logged in as: geekyrakshit. Use `wandb login --relogin` to force relogin


In [3]:
pre_transform = T.NormalizeScale()
transform = T.SamplePoints(2048)
train_dataset = ModelNet(
    root="ModelNet40",
    name='40',
    train=True,
    transform=transform,
    pre_transform=pre_transform
)
val_dataset = ModelNet(
    root="ModelNet40",
    name='40',
    train=False,
    transform=transform,
    pre_transform=pre_transform
)

In [4]:
table = wandb.Table(columns=["Model", "Class", "Split"])
category_dict = {key: 0 for key in config.categories}
for idx in tqdm(range(len(train_dataset))):
    point_cloud = wandb.Object3D(train_dataset[idx].pos.numpy())
    category = config.categories[int(train_dataset[idx].y.item())]
    category_dict[category] += 1
    table.add_data(point_cloud, category, "Train")

wandb.log({"ModelNet40": table})


data = [[key, category_dict[key]] for key in config.categories]
wandb.log({
    "ModelNet40 Class-Frequency Distribution" : wandb.plot.bar(
        wandb.Table(data=data, columns = ["Class", "Frequency"]),
        "Class", "Frequency", title="ModelNet40 Class-Frequency Distribution"
    )
})

In [5]:
table = wandb.Table(columns=["Model", "Class", "Split"])
category_dict = {key: 0 for key in config.categories}
for idx in tqdm(range(len(val_dataset))):
    point_cloud = wandb.Object3D(val_dataset[idx].pos.numpy())
    category = config.categories[int(val_dataset[idx].y.item())]
    category_dict[category] += 1
    table.add_data(point_cloud, category, "Test")

wandb.log({"ModelNet40": table})

data = [[key, category_dict[key]] for key in config.categories]
wandb.log({
    "ModelNet40 Class-Frequency Distribution" : wandb.plot.bar(
        wandb.Table(data=data, columns = ["Class", "Frequency"]),
        "Class", "Frequency", title="ModelNet40 Class-Frequency Distribution"
    )
})

  0%|          | 0/2468 [00:00<?, ?it/s]

In [6]:
wandb.finish()